<a href="https://colab.research.google.com/github/schenita/info5731/blob/master/Floyd_selfstudy12_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [74]:
# Schenita Floyd
# Self Study 12 with TensorFlow
# Fixed imbalance by splitting and merging file
!wget https://raw.githubusercontent.com/wshuyi/demo-customer-churn-ann/master/customer_churn.csv

--2019-04-23 03:46:31--  https://raw.githubusercontent.com/wshuyi/demo-customer-churn-ann/master/customer_churn.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 684858 (669K) [text/plain]
Saving to: ‘customer_churn.csv.1’

customer_churn.csv. 100%[===================>] 668.81K  --.-KB/s    in 0.06s   

2019-04-23 03:46:32 (10.6 MB/s) - ‘customer_churn.csv.1’ saved [684858/684858]



In [0]:
import pandas as pd

In [0]:
df = pd.read_csv("customer_churn.csv")

In [77]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [0]:
!pip install -q tensorflow==2.0.0-alpha0

In [0]:
import tensorflow as tf
from tensorflow import keras

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
train, test = train_test_split(df, test_size=0.2, random_state=1)

In [82]:
len(train)

8000

In [83]:
len(train[train.Exited == 1])

1622

In [84]:
len(train[train.Exited == 0])

6378

In [85]:
#Fixing unbalanced data
target_1_count, target_0_count=train.iloc[:,-1].value_counts()
target_0 = train[train.iloc[:,-1] == 0]
target_1 = train[train.iloc[:,-1] == 1]
target_1_undersample = target_1.sample(target_0_count)
test_undersample = pd.concat([target_1_undersample, target_0],
                             axis=0)
test_undersample.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
2492,2493,15778155,T'ien,520,Germany,Female,31,3,108914.17,1,1,1,183572.39,1
22,23,15699309,Gerasimov,510,Spain,Female,38,4,0.00,1,1,0,118913.53,1
1448,1449,15585367,Diribe,555,Germany,Female,46,4,120392.99,1,1,0,177719.88,1
5871,5872,15709920,Burke,479,France,Female,33,2,208165.53,1,0,0,50774.81,1
4654,4655,15593976,Swanson,578,Germany,Female,31,5,102088.68,4,0,0,187866.21,1


In [86]:
len(test_undersample)

8000

In [0]:
train, valid = train_test_split(test_undersample, test_size=0.2, random_state=1)

In [88]:
len(train)

6400

In [89]:
len(valid)

1600

In [90]:
valid.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
6559,6560,15773852,Hayes,533,Germany,Male,38,4,70362.52,2,1,1,104189.46,0
7020,7021,15786237,Pickworth,651,France,Male,28,7,0.00,2,1,0,823.96,0
5005,5006,15741032,Tsao,733,France,Male,48,5,0.00,1,0,1,117830.57,0
2902,2903,15804002,Kovalev,691,France,Female,33,1,128306.83,1,1,1,113580.79,0
5150,5151,15614716,Okwudilichukwu,515,France,Female,37,0,196853.62,1,1,1,132770.11,0


In [0]:
from tensorflow import feature_column

In [0]:
tf.random.set_seed(1)

In [93]:
df.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [0]:
numeric_columns = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']
categorical_columns = ['Geography',
       'Gender', 'HasCrCard',
       'IsActiveMember']

In [0]:
feature_columns = []

In [0]:
for header in numeric_columns:
  feature_columns.append(feature_column.numeric_column(header))

In [97]:
feature_columns

[NumericColumn(key='CreditScore', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Tenure', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Balance', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='NumOfProducts', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='EstimatedSalary', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [0]:
def get_one_hot_rom_categorical(colname):
  categorical = feature_column.categorical_column_with_vocabulary_list(
  colname,
  train[colname].unique().tolist())
  return feature_column.indicator_column(categorical)

In [0]:
# train["Geography"].unique().tolist()

In [100]:
get_one_hot_rom_categorical("Geography")

IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='Geography', vocabulary_list=('France', 'Spain', 'Germany'), dtype=tf.string, default_value=-1, num_oov_buckets=0))

In [0]:
for col in categorical_columns:
  feature_columns.append(get_one_hot_rom_categorical(col))

In [102]:
feature_columns

[NumericColumn(key='CreditScore', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Tenure', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Balance', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='NumOfProducts', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='EstimatedSalary', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='Geography', vocabulary_list=('France', 'Spain', 'Germany'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='Gender', vocabulary_list=('Male', 'Female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(c

In [0]:
from tensorflow.keras import layers

In [0]:
feature_layer = layers.DenseFeatures(feature_columns)

In [105]:
feature_layer

In [0]:
model = keras.Sequential()
model.add(feature_layer)
model.add(layers.Dense(200, activation='relu'))
model.add(layers.Dense(100, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [0]:
model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['accuracy'])

In [0]:
def df_to_tfdata(df, shuffle=True, bs=32):
  df = df.copy()
  labels = df.pop('Exited')
  ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(df), seed=1)
  ds = ds.batch(bs)
  return ds

In [0]:
train_ds = df_to_tfdata(train)

In [0]:
valid_ds = df_to_tfdata(valid, shuffle=False)
test_ds = df_to_tfdata(test, shuffle=False)

In [111]:
model.fit(train_ds,
         validation_data = valid_ds,
         epochs=5)

Epoch 1/5
200/200 [==============================] - 4s 20ms/step - loss: 5.6023 - accuracy: 0.4327 - val_loss: 3.0753 - val_accuracy: 0.8006
Epoch 2/5
200/200 [==============================] - 4s 19ms/step - loss: 3.1404 - accuracy: 0.7952 - val_loss: 3.0753 - val_accuracy: 0.8006
Epoch 3/5
200/200 [==============================] - 4s 19ms/step - loss: 3.1404 - accuracy: 0.7952 - val_loss: 3.0753 - val_accuracy: 0.8006
Epoch 4/5
200/200 [==============================] - 4s 19ms/step - loss: 3.1404 - accuracy: 0.7952 - val_loss: 3.0753 - val_accuracy: 0.8006
Epoch 5/5
200/200 [==============================] - 4s 19ms/step - loss: 3.1404 - accuracy: 0.7952 - val_loss: 3.0753 - val_accuracy: 0.8006


In [112]:
model.evaluate(test_ds)

63/63 [==============================] - 1s 11ms/step - loss: 3.1829 - accuracy: 0.7925


[3.1829258591409713, 0.7925]

In [113]:
train.Exited.value_counts()

0    5097
1    1303
Name: Exited, dtype: int64

In [114]:
test.Exited.value_counts()

0    1585
1     415
Name: Exited, dtype: int64

In [116]:
415/1585

0.2618296529968454